In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import datetime
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import roc_curve
import os

In [2]:
cv=LeaveOneOut()

In [3]:
#X=pd.read_csv("Kmer4.csv",header=None)
y=pd.read_csv("label.csv",header=None)
y=np.array(y)
#print(X.shape)

In [ ]:
def my_classifier1(X,y):
    print(datetime.datetime.now())
    param1=["best","random"]
    param_grid={"splitter":param1}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["splitter"]
def my_classifier2(p1,X,y):
    print(datetime.datetime.now())
    param2=np.array(range(3,10,1))
    param_grid={"max_depth":param2}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
    return grid_search.best_params_["max_depth"]
def my_classifier(p1,p2,X, y):
    print(datetime.datetime.now())
   # param2=["best","random"]   ##设置splitter范围
   # param3=np.array(range(3,10,1))   ##设置max_depth范围
    i=X.shape[1]
    param3=np.array(range(10,i,100))  ## 设置max_features范围
    param_grid={"max_features":param3}
    grid_search=GridSearchCV(DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2),param_grid,scoring="accuracy",cv=cv)
    grid_search.fit(X,y)
#     print(datetime.datetime.now())
#     print("best parameters:{}".format(grid_search.best_params_))                        #输出最优网格化参数
#     print("best cross-validation score:{:.3f}".format(grid_search.best_score_))         #成员提供优化过程中观察到的最好评分
#     print("best estimator:\n{}".format(grid_search.best_estimator_))
    Predict_Y=[]    #存放预测值
    Prob_Y=[]       #存放预测概率
    Really_Y=y  #存放真实标签
    for train_index,test_index in cv.split(X):
        estimator=DecisionTreeClassifier(random_state=10,splitter=p1,max_depth=p2,max_features=grid_search.best_params_["max_features"])
        estimator.fit(X[train_index],y[train_index])
        Predict_Y.append(estimator.predict(X[test_index])[0])
        Prob_Y.append(estimator.predict_proba(X[test_index]))
    Prob_Y=np.array(Prob_Y)  #将Prob_Y改为numpy数组形式
    Prob_Y=Prob_Y.reshape(173,2)  #原本是(173,1,2)形状，改为（173,2）形状
    Prob_Y=pd.DataFrame(Prob_Y)
    Prob_Y.columns=["0","1"]
    confusion_matrix(Really_Y,Predict_Y)
    TN=confusion_matrix(Really_Y,Predict_Y)[0,0]
    FN=confusion_matrix(Really_Y,Predict_Y)[1,0]
    FP=confusion_matrix(Really_Y,Predict_Y)[0,1]
    TP=confusion_matrix(Really_Y,Predict_Y)[1,1]
    ACC=accuracy_score(Really_Y,Predict_Y)
    F1_score=f1_score(Really_Y,Predict_Y)
    AUC=roc_auc_score(Really_Y,np.array(Prob_Y["1"]))
    Recall=recall_score(Really_Y,Predict_Y)
    Precision=precision_score(Really_Y,Predict_Y)
    S=p1
    D=p2
    M=grid_search.best_params_["max_features"]
#     print("ACC:",ACC)
#     print("f1_score:",F1_score)
#     print("AUC:",AUC)
    return TN,FN,FP,TP,ACC,Recall,Precision,F1_score,AUC,S,D,M

In [7]:
file_path = 'E:\datatest\PseDPC\d9\d91'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
911.csv
**************************************************
2022-02-24 22:27:57.560888
2022-02-24 22:28:03.166923
2022-02-24 22:28:29.259107
##################################################
912.csv
**************************************************
2022-02-24 22:28:48.384908
2022-02-24 22:28:54.299085
2022-02-24 22:29:22.577454
##################################################
913.csv
**************************************************
2022-02-24 22:29:44.312275
2022-02-24 22:29:50.592474
2022-02-24 22:30:00.481015
##################################################
914.csv
**************************************************
2022-02-24 22:30:10.706689
2022-02-24 22:30:16.901084
2022-02-24 22:30:50.347598
##################################################
915.csv
**************************************************
2022-02-24 22:31:10.583458
2022-02-24 22:31:16.854700
2022-02-24 22:31:47.912583
#############################################

In [8]:
file_path = 'E:\datatest\PseDPC\d9\d92'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
921.csv
**************************************************
2022-02-24 22:34:10.669638
2022-02-24 22:34:16.061240
2022-02-24 22:34:42.095570
##################################################
922.csv
**************************************************
2022-02-24 22:34:59.443166
2022-02-24 22:35:04.926474
2022-02-24 22:35:31.324866
##################################################
923.csv
**************************************************
2022-02-24 22:35:51.636501
2022-02-24 22:35:57.546710
2022-02-24 22:36:06.771009
##################################################
924.csv
**************************************************
2022-02-24 22:36:16.327462
2022-02-24 22:36:22.280513
2022-02-24 22:36:52.302212
##################################################
925.csv
**************************************************
2022-02-24 22:37:10.331952
2022-02-24 22:37:16.148417
2022-02-24 22:37:46.633827
#############################################

In [9]:
file_path = 'E:\datatest\PseDPC\d9\d93'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
931.csv
**************************************************
2022-02-24 22:40:00.685172
2022-02-24 22:40:05.701779
2022-02-24 22:40:30.041664
##################################################
932.csv
**************************************************
2022-02-24 22:40:50.263526
2022-02-24 22:40:55.111555
2022-02-24 22:41:19.489333
##################################################
933.csv
**************************************************
2022-02-24 22:41:39.658398
2022-02-24 22:41:44.579238
2022-02-24 22:42:09.352923
##################################################
934.csv
**************************************************
2022-02-24 22:42:29.410259
2022-02-24 22:42:34.705093
2022-02-24 22:43:00.886046
##################################################
935.csv
**************************************************
2022-02-24 22:43:22.620028
2022-02-24 22:43:28.056380
2022-02-24 22:43:57.092666
#############################################

In [10]:
file_path = 'E:\datatest\PseDPC\d9\d94'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
941.csv
**************************************************
2022-02-24 22:45:43.524929
2022-02-24 22:45:49.188754
2022-02-24 22:46:15.568177
##################################################
942.csv
**************************************************
2022-02-24 22:46:37.263131
2022-02-24 22:46:42.623788
2022-02-24 22:47:08.321035
##################################################
943.csv
**************************************************
2022-02-24 22:47:29.049576
2022-02-24 22:47:35.031572
2022-02-24 22:48:04.345143
##################################################
944.csv
**************************************************
2022-02-24 22:48:22.202388
2022-02-24 22:48:28.162420
2022-02-24 22:48:57.561790
##################################################
945.csv
**************************************************
2022-02-24 22:49:15.420980
2022-02-24 22:49:21.372089
2022-02-24 22:49:50.542014
#############################################

In [11]:
file_path = 'E:\datatest\PseDPC\d9\d95'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
951.csv
**************************************************
2022-02-24 22:54:04.596293
2022-02-24 22:54:09.949970
2022-02-24 22:54:36.758272
##################################################
952.csv
**************************************************
2022-02-24 22:54:58.034320
2022-02-24 22:55:03.684204
2022-02-24 22:55:31.313283
##################################################
953.csv
**************************************************
2022-02-24 22:55:53.602648
2022-02-24 22:55:59.264499
2022-02-24 22:56:08.577582
##################################################
954.csv
**************************************************
2022-02-24 22:56:16.862438
2022-02-24 22:56:22.749692
2022-02-24 22:56:52.340495
##################################################
955.csv
**************************************************
2022-02-24 22:57:10.163836
2022-02-24 22:57:16.078007
2022-02-24 22:57:45.577060
#############################################

In [12]:
file_path = 'E:\datatest\PseDPC\d9\d96'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
961.csv
**************************************************
2022-02-24 23:01:36.608936
2022-02-24 23:01:42.255828
2022-02-24 23:02:10.048468
##################################################
962.csv
**************************************************
2022-02-24 23:02:30.708228
2022-02-24 23:02:36.521661
2022-02-24 23:03:05.764401
##################################################
963.csv
**************************************************
2022-02-24 23:03:27.072392
2022-02-24 23:03:32.581652
2022-02-24 23:04:00.299493
##################################################
964.csv
**************************************************
2022-02-24 23:04:18.274401
2022-02-24 23:04:24.197554
2022-02-24 23:04:53.849221
##################################################
965.csv
**************************************************
2022-02-24 23:05:17.416168
2022-02-24 23:05:23.482936
2022-02-24 23:05:33.854189
#############################################

In [13]:
file_path = 'E:\datatest\PseDPC\d9\d97'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
971.csv
**************************************************
2022-02-24 23:07:37.157168
2022-02-24 23:07:42.807053
2022-02-24 23:08:10.752285
##################################################
972.csv
**************************************************
2022-02-24 23:08:29.115155
2022-02-24 23:08:34.661315
2022-02-24 23:09:04.865505
##################################################
973.csv
**************************************************
2022-02-24 23:09:30.541943
2022-02-24 23:09:36.495017
2022-02-24 23:10:06.621414
##################################################
974.csv
**************************************************
2022-02-24 23:10:28.875872
2022-02-24 23:10:34.367208
2022-02-24 23:11:02.476972
##################################################
975.csv
**************************************************
2022-02-24 23:11:25.865396
2022-02-24 23:11:31.950117
2022-02-24 23:12:06.243871
#############################################

In [14]:
file_path = 'E:\datatest\PseDPC\d9\d98'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
981.csv
**************************************************
2022-02-24 23:16:09.537301
2022-02-24 23:16:15.069477
2022-02-24 23:16:41.651357
##################################################
982.csv
**************************************************
2022-02-24 23:17:04.107276
2022-02-24 23:17:09.477936
2022-02-24 23:17:36.132592
##################################################
983.csv
**************************************************
2022-02-24 23:17:54.034696
2022-02-24 23:17:59.534016
2022-02-24 23:18:26.358236
##################################################
984.csv
**************************************************
2022-02-24 23:18:44.230425
2022-02-24 23:18:49.703753
2022-02-24 23:19:16.423287
##################################################
985.csv
**************************************************
2022-02-24 23:19:37.107957
2022-02-24 23:19:43.266474
2022-02-24 23:20:14.363248
#############################################

In [15]:
file_path = 'E:\datatest\PseDPC\d9\d99'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
991.csv
**************************************************
2022-02-24 23:24:40.617533
2022-02-24 23:24:46.077924
2022-02-24 23:25:13.045799
##################################################
992.csv
**************************************************
2022-02-24 23:25:30.161007
2022-02-24 23:25:36.089120
2022-02-24 23:26:06.062924
##################################################
993.csv
**************************************************
2022-02-24 23:26:24.109641
2022-02-24 23:26:30.128537
2022-02-24 23:27:00.709746
##################################################
994.csv
**************************************************
2022-02-24 23:27:18.925979
2022-02-24 23:27:24.965847
2022-02-24 23:27:55.642746
##################################################
995.csv
**************************************************
2022-02-24 23:28:14.806472
2022-02-24 23:28:20.988931
2022-02-24 23:28:51.058507
#############################################

In [16]:
file_path = 'E:\datatest\PseDPC\d9\d910'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9101.csv
**************************************************
2022-02-24 23:32:37.618342
2022-02-24 23:32:43.716034
2022-02-24 23:33:13.940141
##################################################
9102.csv
**************************************************
2022-02-24 23:33:32.137482
2022-02-24 23:33:37.850170
2022-02-24 23:34:06.944329
##################################################
9103.csv
**************************************************
2022-02-24 23:34:27.572686
2022-02-24 23:34:35.087674
2022-02-24 23:34:44.965246
##################################################
9104.csv
**************************************************
2022-02-24 23:34:52.682626
2022-02-24 23:34:58.762332
2022-02-24 23:35:29.212863
##################################################
9105.csv
**************************************************
2022-02-24 23:35:48.290819
2022-02-24 23:35:55.107614
2022-02-24 23:36:04.342872
########################################

In [17]:
file_path = 'E:\datatest\PseDPC\d9\d911'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
911.csv
**************************************************
2022-02-24 23:38:48.898604
2022-02-24 23:38:54.567437
2022-02-24 23:39:23.643644
##################################################
912.csv
**************************************************
2022-02-24 23:39:41.919767
2022-02-24 23:39:47.474905
2022-02-24 23:40:15.594650
##################################################
913.csv
**************************************************
2022-02-24 23:40:38.998033
2022-02-24 23:40:44.684819
2022-02-24 23:41:12.591183
##################################################
914.csv
**************************************************
2022-02-24 23:41:35.328323
2022-02-24 23:41:40.869518
2022-02-24 23:42:08.456687
##################################################
915.csv
**************************************************
2022-02-24 23:42:26.498416
2022-02-24 23:42:32.025628
2022-02-24 23:42:59.527071
#############################################

In [18]:
file_path = 'E:\datatest\PseDPC\d9\d912'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9121.csv
**************************************************
2022-02-24 23:46:48.027724
2022-02-24 23:46:54.017670
2022-02-24 23:47:24.889075
##################################################
9122.csv
**************************************************
2022-02-24 23:47:43.036521
2022-02-24 23:47:49.351658
2022-02-24 23:48:20.916175
##################################################
9123.csv
**************************************************
2022-02-24 23:48:39.980168
2022-02-24 23:48:46.354115
2022-02-24 23:48:55.627304
##################################################
9124.csv
**************************************************
2022-02-24 23:49:04.463724
2022-02-24 23:49:10.843594
2022-02-24 23:49:20.166650
##################################################
9125.csv
**************************************************
2022-02-24 23:49:29.061851
2022-02-24 23:49:35.173527
2022-02-24 23:49:44.406796
########################################

In [19]:
file_path = 'E:\datatest\PseDPC\d9\d913'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9131.csv
**************************************************
2022-02-24 23:52:30.642034
2022-02-24 23:52:37.066872
2022-02-24 23:53:08.549634
##################################################
9132.csv
**************************************************
2022-02-24 23:53:26.767870
2022-02-24 23:53:33.825010
2022-02-24 23:54:07.207673
##################################################
9133.csv
**************************************************
2022-02-24 23:54:25.517713
2022-02-24 23:54:32.199807
2022-02-24 23:55:04.727778
##################################################
9134.csv
**************************************************
2022-02-24 23:55:23.028814
2022-02-24 23:55:29.719912
2022-02-24 23:56:02.200012
##################################################
9135.csv
**************************************************
2022-02-24 23:56:20.425250
2022-02-24 23:56:26.745340
2022-02-24 23:56:36.042489
########################################

In [20]:
file_path = 'E:\datatest\PseDPC\d9\d914'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9141.csv
**************************************************
2022-02-24 23:59:54.406744
2022-02-25 00:00:00.679981
2022-02-25 00:00:32.151758
##################################################
9142.csv
**************************************************
2022-02-25 00:00:50.393986
2022-02-25 00:00:56.392901
2022-02-25 00:01:27.431858
##################################################
9143.csv
**************************************************
2022-02-25 00:01:46.904756
2022-02-25 00:01:53.534052
2022-02-25 00:02:26.203612
##################################################
9144.csv
**************************************************
2022-02-25 00:02:44.589421
2022-02-25 00:02:51.262567
2022-02-25 00:03:24.172517
##################################################
9146.csv
**************************************************
2022-02-25 00:03:42.558326
2022-02-25 00:03:48.912353
2022-02-25 00:04:20.367169
########################################

In [21]:
file_path = 'E:\datatest\PseDPC\d9\d915'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9151.csv
**************************************************
2022-02-25 00:07:26.573973
2022-02-25 00:07:32.998783
2022-02-25 00:08:04.545380
##################################################
9152.csv
**************************************************
2022-02-25 00:08:22.781590
2022-02-25 00:08:28.953107
2022-02-25 00:08:58.750379
##################################################
9153.csv
**************************************************
2022-02-25 00:09:17.230938
2022-02-25 00:09:23.742517
2022-02-25 00:09:33.480435
##################################################
9154.csv
**************************************************
2022-02-25 00:09:41.996678
2022-02-25 00:09:48.501247
2022-02-25 00:09:58.289088
##################################################
9155.csv
**************************************************
2022-02-25 00:10:06.819237
2022-02-25 00:10:13.053558
2022-02-25 00:10:22.539179
########################################

In [22]:
file_path = 'E:\datatest\PseDPC\d9\d916'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9161.csv
**************************************************
2022-02-25 00:12:58.647546
2022-02-25 00:13:04.461956
2022-02-25 00:13:33.142222
##################################################
9162.csv
**************************************************
2022-02-25 00:13:51.406384
2022-02-25 00:13:57.786287
2022-02-25 00:14:29.462564
##################################################
9163.csv
**************************************************
2022-02-25 00:14:51.691065
2022-02-25 00:14:58.090943
2022-02-25 00:15:29.742259
##################################################
9164.csv
**************************************************
2022-02-25 00:15:51.951837
2022-02-25 00:15:58.364710
2022-02-25 00:16:29.981112
##################################################
9165.csv
**************************************************
2022-02-25 00:16:52.169752
2022-02-25 00:16:57.903412
2022-02-25 00:17:26.482920
########################################

In [23]:
file_path = 'E:\datatest\PseDPC\d9\d917'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9171.csv
**************************************************
2022-02-25 00:21:40.729684
2022-02-25 00:21:46.478331
2022-02-25 00:22:15.267301
##################################################
9172.csv
**************************************************
2022-02-25 00:22:37.513787
2022-02-25 00:22:44.007385
2022-02-25 00:23:15.756468
##################################################
9173.csv
**************************************************
2022-02-25 00:23:34.083407
2022-02-25 00:23:40.629922
2022-02-25 00:24:12.566473
##################################################
9174.csv
**************************************************
2022-02-25 00:24:30.793679
2022-02-25 00:24:37.325238
2022-02-25 00:25:09.084233
##################################################
9175.csv
**************************************************
2022-02-25 00:25:27.235668
2022-02-25 00:25:33.747247
2022-02-25 00:26:05.368666
########################################

In [24]:
file_path = 'E:\datatest\PseDPC\d9\d918'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9181.csv
**************************************************
2022-02-25 00:30:16.736137
2022-02-25 00:30:22.689184
2022-02-25 00:30:52.980168
##################################################
9182.csv
**************************************************
2022-02-25 00:31:15.440050
2022-02-25 00:31:21.756179
2022-02-25 00:31:53.517175
##################################################
9183.csv
**************************************************
2022-02-25 00:32:15.599095
2022-02-25 00:32:21.931153
2022-02-25 00:32:53.719104
##################################################
9184.csv
**************************************************
2022-02-25 00:33:15.622502
2022-02-25 00:33:22.042360
2022-02-25 00:33:53.709623
##################################################
9185.csv
**************************************************
2022-02-25 00:34:15.577094
2022-02-25 00:34:21.913142
2022-02-25 00:34:53.610365
########################################

In [25]:
file_path = 'E:\datatest\PseDPC\d9\d919'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9191.csv
**************************************************
2022-02-25 00:39:12.536581
2022-02-25 00:39:18.216384
2022-02-25 00:39:47.075201
##################################################
9192.csv
**************************************************
2022-02-25 00:40:10.028761
2022-02-25 00:40:15.984825
2022-02-25 00:40:45.692371
##################################################
9193.csv
**************************************************
2022-02-25 00:41:07.433176
2022-02-25 00:41:13.055134
2022-02-25 00:41:41.882042
##################################################
9194.csv
**************************************************
2022-02-25 00:42:03.575966
2022-02-25 00:42:09.246821
2022-02-25 00:42:38.041785
##################################################
9195.csv
**************************************************
2022-02-25 00:42:59.791562
2022-02-25 00:43:06.100703
2022-02-25 00:43:37.894644
########################################

In [26]:
file_path = 'E:\datatest\PseDPC\d9\d920'
#file_path2='E:\datatest\Kmer\results'
files = os.listdir(file_path)
#results = []
transfer=StandardScaler()
#csv_path = r'E:\test'
mark = np.array([])
for file in files:
    results=[]
    print('#'*50)
    path_whole = file_path + '\\' + file
    a=pd.read_csv(path_whole,header=None)
    #a=np.array(a)
    x=transfer.fit_transform(a) 
    print(file)
    print('*'*50)
    x1,r1=my_classifier1(x,y)
    x2,r2=my_classifier2(x1,x,y)
    r3= my_classifier(x1,x2,x, y)
    # 结果保存
   # print(r1.shape)
    #print(r2.shape)
   # print(r3.shape)
    temp_r = np.zeros((10, 1))
    # TN_a,FN_a,FP_a,TP_a,accuracy_a,pre_a,rec_a,f1_a,auroc_a
    mark = np.array([[file, 'TN', 'FN', 'FP', 'TP', 'Acc','Pre','Rec','F','AUROC']])
    mark = mark.reshape((10, 1))
    #print(mark.shape)
    # print(mark)
    final_r = np.hstack((mark, r1,temp_r, r2, temp_r, r3))
    # print(final_r.shape)
    final_pd = pd.DataFrame(final_r)
    final_pd.to_csv(file_path+'\\result.csv', mode='a', index=False, header=False)
    

##################################################
9201.csv
**************************************************
2022-02-25 00:47:53.011077
2022-02-25 00:47:58.501364
2022-02-25 00:48:25.861164
##################################################
9202.csv
**************************************************
2022-02-25 00:48:46.887929
2022-02-25 00:48:52.789118
2022-02-25 00:49:21.767608
##################################################
9203.csv
**************************************************
2022-02-25 00:49:45.163020
2022-02-25 00:49:51.050241
2022-02-25 00:50:20.632095
##################################################
9204.csv
**************************************************
2022-02-25 00:50:45.080683
2022-02-25 00:50:50.868198
2022-02-25 00:51:20.157834
##################################################
9205.csv
**************************************************
2022-02-25 00:51:44.541596
2022-02-25 00:51:50.451810
2022-02-25 00:52:19.374422
########################################